# Correlation Analysis
*Master "Applied Data Science" @ Nordakademie*
*Modul: Text Analytics*

### General pre-work

In [12]:
# Import necessary libraries

# Overall needed
import pandas as pd # to work with data frames

# Needed for visualisations and smaller functions such as time tracking or saving files

import csv
import os # to define a dedicated output folder

In [13]:
output_folder = 'data/analysis' # refer to a new folder to store only sentiment result files

# check if folder can be found, else create it
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Add the normal distribution stock values to the combined sentiment table

csv_file_path_normal_3 = 'data/transfer/cleaned_articles_normalverteilt_3.csv' # define the path of the input csv file with 3 categories, normal distribution

# Load the data set
df_3categories_normal = pd.read_csv(csv_file_path_normal_3, encoding='utf-8', quoting=csv.QUOTE_ALL) # ensuring the right encoding as in the csv file we still encounter incorrectly encoded special characters

csv_file_path_combined_sentiments = 'data/sentiment_results/sentiment_results_combined.csv' # define the path of the input csv file with the sentiment results

# Load the data set
df_combined_sentiments = pd.read_csv(csv_file_path_combined_sentiments, encoding='utf-8', quoting=csv.QUOTE_ALL) # ensuring the right encoding as in the csv file we still encounter incorrectly encoded special characters

## Pre-Processing

In [14]:
# Merge the two data frames based on a common column, such as an index or a specific column name
# In this example, I assume you want to use the index as the merge key
merged_df = df_combined_sentiments.merge(df_3categories_normal[['Stock_ValueStd']], left_index=True, right_index=True)

# Save the merged data frame to a new variable or file if needed
# merged_df contains the combined data with the "Stock_ValueStd" column added

# If you want to save the merged data frame to a new CSV file, you can use the following:
# merged_df.to_csv('merged_data.csv', index=False)

print(merged_df.head(3))

   Row_Number                   Unternehmen Newstyp   Quelle Nearest_Date  \
0           1  Porsche Automobil Holding SE    News  onvista   2021-06-01   
1           2                    Beiersdorf    News  onvista   2021-06-02   
2           3          Heidelberg Materials    News  onvista   2021-06-02   

                                        Cleaned_Text Stock_Value  \
0  neu Rumor Porschebörsengang Sixt Berenberg stu...     neutral   
1  Beiersdorf Aktie Kaufempfehlung beflügeln Bere...    positive   
2  Heidelbergcement klimaneutral Zementwerk Weg B...    positive   

   TextBlob_Sentiment_Score TextBlob_Evenly_Separated_Label  \
0                   -0.0500                        Negative   
1                    0.2000                        Positive   
2                   -0.0875                        Negative   

  TextBlob_Normal_Distribution_Label  NLTK_Sentiment  \
0                            Neutral         -0.3612   
1                           Positive          0.0000 